In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:
import os
from openai import OpenAI

anthropic_client = OpenAI(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    base_url="https://api.anthropic.com/v1/"
)

In [5]:
messages = [
    {"role": "user", "content": "hello"}
]

response = anthropic_client.chat.completions.create(
    model='claude-sonnet-4-5',
    messages=messages
)

print(response.choices[0].message.content)

Hello! How can I help you today?


In [6]:
import rag
index = rag.initialize_index()

Indexed 385 chunks from 95 documents


In [9]:
class ChatCompletionsRAG(rag.RAG):

    def llm(self, user_prompt):
        messages = [
            {"role": "system", "content": self.rag_instructions},
            {"role": "user", "content": user_prompt}
        ]

        response = self.llm_client.chat.completions.create(
            model=self.model_name,
            messages=messages,
        )

        return response.choices[0].message.content


In [10]:
responses_rag = ChatCompletionsRAG(
    index,
    anthropic_client,
    model_name='claude-sonnet-4-5'
)

response = responses_rag.rag("llm as a judge")
print(response[:50] + "...")

# LLM as a Judge

Based on the documentation, **LL...


In [11]:
!uv add anthropic

Resolved 157 packages in 24ms
Audited 135 packages in 0.70ms


In [12]:
from anthropic import Anthropic

In [13]:
anthropic_client = Anthropic()

In [14]:
messages = [
    {"role": "user", "content": "hello"}
]

message = anthropic_client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    messages=messages
)

In [16]:
print(message.model_dump_json(indent=2))

{
  "id": "msg_01DmbH1jxUT3ab5Gpd2odGvq",
  "content": [
    {
      "citations": null,
      "text": "Hello! How can I help you today?",
      "type": "text"
    }
  ],
  "model": "claude-sonnet-4-5-20250929",
  "role": "assistant",
  "stop_reason": "end_turn",
  "stop_sequence": null,
  "type": "message",
  "usage": {
    "cache_creation": {
      "ephemeral_1h_input_tokens": 0,
      "ephemeral_5m_input_tokens": 0
    },
    "cache_creation_input_tokens": 0,
    "cache_read_input_tokens": 0,
    "input_tokens": 8,
    "output_tokens": 12,
    "server_tool_use": null,
    "service_tier": "standard"
  }
}


In [20]:
print(message.content[0].text)

Hello! How can I help you today?


In [22]:
messages = [
    {"role": "user", "content": "Hello"}
]

instructions = "You are a helpful assistant. Reply with emojis."

message = anthropic_client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    system=instructions,
    messages=messages
)

print(message.content[0].text)

Hello! 👋😊 How can I help you today? ✨


In [23]:
instructions = """
You're an assistant that can make jokes. Always find out the name of
the person to make the jokes personalized. Once you know the name,
make the joke about them.
""".strip()

messages = [
    {"role": "user", "content": "tell me a joke"}
]

message = anthropic_client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    system=instructions,
    messages=messages
)

print(message.content[0].text)

Hi there! I'd love to tell you a joke, but first - what's your name? I want to make it extra special and personalized just for you! 😄


In [24]:
messages.append({"role": "assistant", "content": message.content[0].text})
messages.append({"role": "user", "content": "Alexey"})

In [25]:
message = anthropic_client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    system=instructions,
    messages=messages
)

print(message.content[0].text)

Great to meet you, Alexey! Here's one for you:

Why did Alexey bring a ladder to the bar?

Because he heard the drinks were on the house, and knowing Alexey, he wasn't about to let a technicality stop him from getting a good deal! 🍻😄

Want another one, or should I quit while I'm ahead?


In [26]:
messages = [
    {"role": "user", "content": "Tell me a bedtime story (100 words)"}
]

with anthropic_client.messages.stream(
    max_tokens=1024,
    messages=messages,
    model="claude-sonnet-4-5",
) as stream:
  for text in stream.text_stream:
      print(text, end="", flush=True)

# The Sleepy Star

High above a quiet village, a tiny star yawned. She had twinkled all night and was very tired.

"Time for bed," whispered the Moon.

The little star drifted down through cotton candy clouds and snuggled into a child's dream. There, she became a nightlight, glowing softly beside a sleeping boy.

The boy smiled in his sleep, feeling safe and warm.

When morning came, the star floated back to the sky, ready to rest among her sister stars.

And every night since, she watches over that same boy, twinkling gently until dawn.

*Sweet dreams.*

In [27]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

In [33]:
instructions = "Extract the event information."
content = "Alice and Bob are going to a science fair on Friday."

messages = [{"role": "user", "content": content}]

response = anthropic_client.beta.messages.parse(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    betas=["structured-outputs-2025-11-13"],
    messages=messages,
    output_format=CalendarEvent,
)

In [31]:
print(response.parsed_output)

name='science fair' date='Friday' participants=['Alice', 'Bob']


In [34]:
structured_output_tool = {
    "name": "final_result",
    "description": "Tool for structured output",
    "input_schema": CalendarEvent.model_json_schema()
}

In [35]:
response = anthropic_client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    messages=messages,
    tools=[structured_output_tool],
    tool_choice={"type": "tool", "name": structured_output_tool['name']}
)

In [42]:
event = CalendarEvent.model_validate(response.content[0].input)
print(event)

name='Science fair' date='Friday' participants=['Alice', 'Bob']


In [43]:
class AnthropicRAG(rag.RAG):

    def llm(self, user_prompt):
        messages = [{"role": "user", "content": user_prompt}]

        response = self.llm_client.beta.messages.parse(
            model=self.model_name,
            max_tokens=1024,
            betas=["structured-outputs-2025-11-13"],
            system=self.rag_instructions,
            messages=messages,
            output_format=self.output_type,
        )

        return response.parsed_output

In [44]:
anthropic_rag = AnthropicRAG(
    index,
    anthropic_client,
    model_name='claude-sonnet-4-5'
)

response = anthropic_rag.rag("llm as a judge")
print(response.answer[:50] + "...")

# LLM as a Judge

LLM as a judge is an evaluation ...
